# Welcome back

Welcome back! In our fifth session, we will:

* Fit some linear models with statsmodels
* Fit the same and a K-nearest neighbors model with scikit-learn

# Preliminaries

In [1]:
import numpy as np
import pandas as pd

In [2]:
def fetch_triangle(line, url_stem = 'https://www.pirategrunt.com/paw/'):
  url = url_stem + line + '_pos.csv'
  df_out = pd.read_csv(url)
  df_out.columns = [
    'group_code', 
    'group_name', 
    'accident_year', 
    'development_year', 
    'lag',
    'cumulative_incurred', 
    'cumulative_paid', 
    'bulk_loss', 
    'earned_premium',
    'earned_premium_ceded',
    'earned_premium_net', 
    'single',
    'posted_reserve_97']
  df_out['line'] = line
  return df_out


In [3]:
def augment_triangle(df_triangle):
  """
  Add prior cumulative, incremental and ldf columns for paid and incurred
  """
  df_triangle = df_triangle.set_index(['line', 'group_code', 'accident_year', 'lag'])
  group_cols = ['group_code', 'accident_year']
  df_triangle[
      ['prior_cumulative_paid', 'prior_cumulative_incurred']
    ] = df_triangle.groupby(group_cols)[['cumulative_paid', 'cumulative_incurred']].shift()
  df_triangle['incremental_paid'] = df_triangle.cumulative_paid - df_triangle.prior_cumulative_paid
  df_triangle['incremental_incurred'] = df_triangle.cumulative_incurred - df_triangle.prior_cumulative_incurred
  df_triangle['ldf_paid'] = df_triangle['cumulative_paid'] / df_triangle['prior_cumulative_paid']
  df_triangle['ldf_incurred'] = df_triangle['cumulative_incurred'] / df_triangle['prior_cumulative_incurred']
  # This will be important later
  df_triangle = df_triangle.replace([np.inf, -np.inf], np.nan)
  return df_triangle


In [4]:
df_allstate = fetch_triangle('wkcomp')

We could run these two statements in either order.

In [5]:
df_allstate = augment_triangle(df_allstate)
df_allstate = df_allstate.query('group_name == "Allstate Ins Co Grp"').copy()

## Two ways of thinking about modeling

* `statsmodels`
    * Textbook implementations
    * Lovely diasnostics, from Akaike Information Criteria to Cook's distance
* `scikit learn`
    * Performance on a test set is the only diagnostic we need
    * Unity of interface means loads of models. 

# `statsmodels`

We can remember this with the easy pnuemonic: sm = Stephen Mildenhall

In [6]:
import statsmodels.api as sm

This uses a vector for Y and a (design) matrix for X. The OLS method will return a model object which we can later use to come up with a fit.

In [7]:
mdl_ols = sm.OLS(
  df_allstate.incremental_paid, 
  df_allstate.prior_cumulative_paid
)

MissingDataError: exog contains inf or nans

D'oh! What went wrong? We have some nans in our data. We can eliminate them by setting `missing = 'drop'`.

In [8]:
mdl_ols = sm.OLS(
  df_allstate.incremental_paid, 
  df_allstate.prior_cumulative_paid, 
  missing = 'drop'
)

In [9]:
fit_ols = mdl_ols.fit()

fit_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       incremental_paid   R-squared (uncentered):                   0.195
Model:                            OLS   Adj. R-squared (uncentered):              0.186
Method:                 Least Squares   F-statistic:                              21.57
Date:                Tue, 24 Aug 2021   Prob (F-statistic):                    1.17e-05
Time:                        10:04:46   Log-Likelihood:                         -1026.4
No. Observations:                  90   AIC:                                      2055.
Df Residuals:                      89   BIC:                                      2057.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
prior_cumulative_paid     0.0692      0.015      4.645      0.000       0.040       0.099
==============================================================================
Omnibus:                       45.224   Durbin-Watson:                   1.209
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               99.521
Skew:                           1.957   Prob(JB):                     2.45e-22
Kurtosis:                       6.350   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
print(fit_ols.rsquared)
print(fit_ols.params)
print(fit_ols.tvalues)

0.1951055998610457
prior_cumulative_paid    0.069235
dtype: float64
prior_cumulative_paid    4.644729
dtype: float64


The diagnostics aren't great. We'll incorporate the lag, but we'll do it as a categorical variable. How do we do that? Easy, convert it to a string.

In [11]:
df_allstate['lag_cat'] = df_allstate.index.get_level_values('lag').astype(str)

Chain the methods and ignore the intermediate object.

In [12]:
fit_ols_z = sm.OLS(
  df_allstate.incremental_paid, 
  df_allstate[['prior_cumulative_paid', 'lag_cat']], 
  missing = 'drop'
).fit()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

Hmm, that doesn't work. The problem is that `OLS` wants a numeric matrix and we just fed it some character data. What we need is to create a "one-hot encoding" in a design matrix. We could do that from sratch, but this will take effort and could lead to mistakes

There's another way:

In [13]:
import statsmodels.formula.api as smf

fit_ols_2 = smf.ols(
  'incremental_paid ~ prior_cumulative_paid + lag_cat', 
  data = df_allstate.dropna()
).fit()

fit_ols_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       incremental_paid   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.569
Method:                 Least Squares   F-statistic:                     14.07
Date:                Tue, 24 Aug 2021   Prob (F-statistic):           2.63e-13
Time:                        10:05:16   Log-Likelihood:                -976.53
No. Observations:                  90   AIC:                             1973.
Df Residuals:                      80   BIC:                             1998.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept             -1.068e+04   5007.512     -2.133      0.036   -2.06e+04    -714.371
lag_cat[T.2]           5.562e+04   6271.582      8.869      0.000    4.31e+04    6.81e+04
lag_cat[T.3]           3.225e+04   6048.764      5.332      0.000    2.02e+04    4.43e+04
lag_cat[T.4]           1.845e+04   5967.274      3.091      0.003    6572.448    3.03e+04
lag_cat[T.5]           1.124e+04   5937.400      1.892      0.062    -580.133    2.31e+04
lag_cat[T.6]           6842.9401   5925.302      1.155      0.252   -4948.787    1.86e+04
lag_cat[T.7]           4849.6661   5920.294      0.819      0.415   -6932.093    1.66e+04
lag_cat[T.8]           2694.4088   5917.954      0.455      0.650   -9082.696    1.45e+04
lag_cat[T.9]           1870.8194   5917.156      0.316      0.753   -9904.697    1.36e+04
prior_cumulative_paid     0.0745      0.017      4.330      0.000       0.040       0.109
==============================================================================
Omnibus:                       16.513   Durbin-Watson:                   1.369
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               43.380
Skew:                          -0.524   Prob(JB):                     3.80e-10
Kurtosis:                       6.235   Cond. No.                     1.49e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.49e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Let's do that without the intercept.

In [14]:
fit_ols_3 = smf.ols(
  'incremental_paid ~ 0 + prior_cumulative_paid + lag_cat', 
  data = df_allstate.dropna()
).fit()

fit_ols_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       incremental_paid   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.569
Method:                 Least Squares   F-statistic:                     14.07
Date:                Tue, 24 Aug 2021   Prob (F-statistic):           2.63e-13
Time:                        10:05:21   Log-Likelihood:                -976.53
No. Observations:                  90   AIC:                             1973.
Df Residuals:                      80   BIC:                             1998.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
lag_cat[10]           -1.068e+04   5007.512     -2.133      0.036   -2.06e+04    -714.371
lag_cat[2]             4.494e+04   4237.587     10.605      0.000    3.65e+04    5.34e+04
lag_cat[3]             2.158e+04   4443.149      4.856      0.000    1.27e+04    3.04e+04
lag_cat[4]             7768.0627   4627.921      1.679      0.097   -1441.793     1.7e+04
lag_cat[5]              556.0311   4754.800      0.117      0.907   -8906.323       1e+04
lag_cat[6]            -3836.6979   4841.821     -0.792      0.430   -1.35e+04    5798.833
lag_cat[7]            -5829.9719   4900.980     -1.190      0.238   -1.56e+04    3923.289
lag_cat[8]            -7985.2293   4948.363     -1.614      0.111   -1.78e+04    1862.327
lag_cat[9]            -8808.8186   4980.366     -1.769      0.081   -1.87e+04    1102.425
prior_cumulative_paid     0.0745      0.017      4.330      0.000       0.040       0.109
==============================================================================
Omnibus:                       16.513   Durbin-Watson:                   1.369
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               43.380
Skew:                          -0.524   Prob(JB):                     3.80e-10
Kurtosis:                       6.235   Cond. No.                     8.82e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The intercept was subsuming the lag 10 variable. What we REALLY want is an interaction term.

In [15]:
fit_ols_cl = smf.ols(
  'incremental_paid ~ 0 + prior_cumulative_paid:lag_cat', 
  data = df_allstate.dropna()
).fit()

fit_ols_cl.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       incremental_paid   R-squared (uncentered):                   0.853
Model:                            OLS   Adj. R-squared (uncentered):              0.836
Method:                 Least Squares   F-statistic:                              52.10
Date:                Tue, 24 Aug 2021   Prob (F-statistic):                    5.23e-30
Time:                        10:05:31   Log-Likelihood:                         -950.01
No. Observations:                  90   AIC:                                      1918.
Df Residuals:                      81   BIC:                                      1941.
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
prior_cumulative_paid:lag_cat[10]     0.0088      0.016      0.545      0.588      -0.023       0.041
prior_cumulative_paid:lag_cat[2]      1.1569      0.069     16.802      0.000       1.020       1.294
prior_cumulative_paid:lag_cat[3]      0.3283      0.031     10.537      0.000       0.266       0.390
prior_cumulative_paid:lag_cat[4]      0.1466      0.023      6.279      0.000       0.100       0.193
prior_cumulative_paid:lag_cat[5]      0.0865      0.020      4.252      0.000       0.046       0.127
prior_cumulative_paid:lag_cat[6]      0.0540      0.019      2.886      0.005       0.017       0.091
prior_cumulative_paid:lag_cat[7]      0.0401      0.018      2.263      0.026       0.005       0.075
prior_cumulative_paid:lag_cat[8]      0.0263      0.017      1.543      0.127      -0.008       0.060
prior_cumulative_paid:lag_cat[9]      0.0230      0.017      1.386      0.170      -0.010       0.056
==============================================================================
Omnibus:                       31.282   Durbin-Watson:                   2.051
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              533.073
Skew:                           0.071   Prob(JB):                    1.76e-116
Kurtosis:                      14.922   Cond. No.                         4.24
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We can also use patsy to create a design matrix

In [16]:
from patsy import dmatrices

y, X = dmatrices(
  'incremental_paid ~ 0 + prior_cumulative_paid:lag_cat', 
  data = df_allstate.dropna(),
  return_type='dataframe')

X.head()

prior_cumulative_paid:lag_cat[10]  \
line   group_code accident_year lag                                      
wkcomp 86         1988          2                                  0.0   
                                3                                  0.0   
                                4                                  0.0   
                                5                                  0.0   
                                6                                  0.0   

                                     prior_cumulative_paid:lag_cat[2]  \
line   group_code accident_year lag                                     
wkcomp 86         1988          2                             70571.0   
                                3                                 0.0   
                                4                                 0.0   
                                5                                 0.0   
                                6                                 0.0   

                                     prior_cumulative_paid:lag_cat[3]  \
line   group_code accident_year lag                                     
wkcomp 86         1988          2                                 0.0   
                                3                            155905.0   
                                4                                 0.0   
                                5                                 0.0   
                                6                                 0.0   

                                     prior_cumulative_paid:lag_cat[4]  \
line   group_code accident_year lag                                     
wkcomp 86         1988          2                                 0.0   
                                3                                 0.0   
                                4                            220744.0   
                                5                                 0.0   
                                6                                 0.0   

                                     prior_cumulative_paid:lag_cat[5]  \
line   group_code accident_year lag                                     
wkcomp 86         1988          2                                 0.0   
                                3                                 0.0   
                                4                                 0.0   
                                5                            251595.0   
                                6                                 0.0   

                                     prior_cumulative_paid:lag_cat[6]  \
line   group_code accident_year lag                                     
wkcomp 86         1988          2                                 0.0   
                                3                                 0.0   
                                4                                 0.0   
                                5                                 0.0   
                                6                            274156.0   

                                     prior_cumulative_paid:lag_cat[7]  \
line   group_code accident_year lag                                     
wkcomp 86         1988          2                                 0.0   
                                3                                 0.0   
                                4                                 0.0   
                                5                                 0.0   
                                6                                 0.0   

                                     prior_cumulative_paid:lag_cat[8]  \
line   group_code accident_year lag                                     
wkcomp 86         1988          2                                 0.0   
                                3                                 0.0   
                                4                                 0.0   
                                5                                 0.0   
        

In [17]:
fit_ols_cl_2 = sm.OLS(
  y,
  X
).fit()

fit_ols_cl_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       incremental_paid   R-squared (uncentered):                   0.853
Model:                            OLS   Adj. R-squared (uncentered):              0.836
Method:                 Least Squares   F-statistic:                              52.10
Date:                Tue, 24 Aug 2021   Prob (F-statistic):                    5.23e-30
Time:                        10:05:39   Log-Likelihood:                         -950.01
No. Observations:                  90   AIC:                                      1918.
Df Residuals:                      81   BIC:                                      1941.
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
prior_cumulative_paid:lag_cat[10]     0.0088      0.016      0.545      0.588      -0.023       0.041
prior_cumulative_paid:lag_cat[2]      1.1569      0.069     16.802      0.000       1.020       1.294
prior_cumulative_paid:lag_cat[3]      0.3283      0.031     10.537      0.000       0.266       0.390
prior_cumulative_paid:lag_cat[4]      0.1466      0.023      6.279      0.000       0.100       0.193
prior_cumulative_paid:lag_cat[5]      0.0865      0.020      4.252      0.000       0.046       0.127
prior_cumulative_paid:lag_cat[6]      0.0540      0.019      2.886      0.005       0.017       0.091
prior_cumulative_paid:lag_cat[7]      0.0401      0.018      2.263      0.026       0.005       0.075
prior_cumulative_paid:lag_cat[8]      0.0263      0.017      1.543      0.127      -0.008       0.060
prior_cumulative_paid:lag_cat[9]      0.0230      0.017      1.386      0.170      -0.010       0.056
==============================================================================
Omnibus:                       31.282   Durbin-Watson:                   2.051
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              533.073
Skew:                           0.071   Prob(JB):                    1.76e-116
Kurtosis:                      14.922   Cond. No.                         4.24
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Easy to create a model with a different variable.

In [18]:
fit_ols_ep = smf.ols(
  'incremental_paid ~ 0 + earned_premium_net:lag_cat',
  data = df_allstate.dropna()
).fit()

fit_ols_ep.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       incremental_paid   R-squared (uncentered):                   0.905
Model:                            OLS   Adj. R-squared (uncentered):              0.894
Method:                 Least Squares   F-statistic:                              85.32
Date:                Tue, 24 Aug 2021   Prob (F-statistic):                    1.47e-37
Time:                        10:05:44   Log-Likelihood:                         -930.47
No. Observations:                  90   AIC:                                      1879.
Df Residuals:                      81   BIC:                                      1901.
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
earned_premium_net:lag_cat[10]     0.0065      0.010      0.657      0.513      -0.013       0.026
earned_premium_net:lag_cat[2]      0.2144      0.010     21.729      0.000       0.195       0.234
earned_premium_net:lag_cat[3]      0.1321      0.010     13.387      0.000       0.112       0.152
earned_premium_net:lag_cat[4]      0.0787      0.010      7.972      0.000       0.059       0.098
earned_premium_net:lag_cat[5]      0.0515      0.010      5.217      0.000       0.032       0.071
earned_premium_net:lag_cat[6]      0.0343      0.010      3.476      0.001       0.015       0.054
earned_premium_net:lag_cat[7]      0.0260      0.010      2.634      0.010       0.006       0.046
earned_premium_net:lag_cat[8]      0.0188      0.010      1.903      0.061      -0.001       0.038
earned_premium_net:lag_cat[9]      0.0162      0.010      1.641      0.105      -0.003       0.036
==============================================================================
Omnibus:                       41.147   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              146.593
Skew:                           1.442   Prob(JB):                     1.47e-32
Kurtosis:                       8.548   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Hey, earned premium is a better predictor!

In [19]:
print(fit_ols_cl.rsquared)
print(fit_ols_ep.rsquared)

0.8526976159298703
0.9045827396643866


## Predictions

In [20]:
fit_ols_cl.predict(df_allstate.dropna()).groupby('accident_year').sum()
fit_ols_ep.predict(df_allstate.dropna()).groupby('accident_year').sum()

accident_year
1988    228327.203149
1989    216475.349552
1990    162143.074683
1991    181613.894389
1992    146165.919971
1993    116294.505852
1994    100865.694586
1995     84661.220281
1996     53963.242043
1997      4425.501799
dtype: float64

## Fit a GLM

As with R, simply swap out the function and use the same input data.

In [21]:
mdl_pois = sm.GLM(y, X, family=sm.families.Poisson())
fit_pois = mdl_pois.fit()
fit_pois.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:       incremental_paid   No. Observations:                   90
Model:                            GLM   Df Residuals:                       81
Model Family:                 Poisson   Df Model:                            8
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                   -inf
Date:                Tue, 24 Aug 2021   Deviance:                   4.1136e+06
Time:                        10:06:05   Pearson chi2:                 3.01e+08
No. Iterations:                     8                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
prior_cumulative_paid:lag_cat[10]  2.738e-05   3.21e-08    853.411      0.000    2.73e-05    2.74e-05
prior_cumulative_paid:lag_cat[2]      0.0002   2.51e-08   6914.726      0.000       0.000       0.000
prior_cumulative_paid:lag_cat[3]   7.613e-05   1.44e-08   5286.657      0.000    7.61e-05    7.62e-05
prior_cumulative_paid:lag_cat[4]   5.188e-05   1.35e-08   3837.084      0.000    5.19e-05    5.19e-05
prior_cumulative_paid:lag_cat[5]   4.375e-05   1.44e-08   3036.365      0.000    4.37e-05    4.38e-05
prior_cumulative_paid:lag_cat[6]   3.865e-05   1.61e-08   2401.191      0.000    3.86e-05    3.87e-05
prior_cumulative_paid:lag_cat[7]   3.581e-05   1.73e-08   2067.817      0.000    3.58e-05    3.58e-05
prior_cumulative_paid:lag_cat[8]   3.305e-05   2.02e-08   1633.935      0.000     3.3e-05    3.31e-05
prior_cumulative_paid:lag_cat[9]   3.182e-05   2.08e-08   1527.062      0.000    3.18e-05    3.19e-05
=====================================================================================================
"""

In [22]:
np.exp(fit_pois.params)

prior_cumulative_paid:lag_cat[10]    1.000027
prior_cumulative_paid:lag_cat[2]     1.000173
prior_cumulative_paid:lag_cat[3]     1.000076
prior_cumulative_paid:lag_cat[4]     1.000052
prior_cumulative_paid:lag_cat[5]     1.000044
prior_cumulative_paid:lag_cat[6]     1.000039
prior_cumulative_paid:lag_cat[7]     1.000036
prior_cumulative_paid:lag_cat[8]     1.000033
prior_cumulative_paid:lag_cat[9]     1.000032
dtype: float64

<!-- TODO switch this up so that coefficients are on a non-log scale -->

# scikit-learn

![](https://scikit-learn.org/stable/_static/ml_map.png)

`sklearn` is the defacto standard Machine Learning API for Python.  Other libraries yield to the simplicity of its API. 

* Want to do some Keras Deep learning?  No problem, just use `keras.wrappers.scikit_learn`
* XGBoost anyone?  Use: `xgboost.sklearn`
* Don't want to learn the syntax for the Light GBM? `lightgbm.sklearn` to the rescue.
* Natural langauge processing requires unique functionality, right? Nope, `nltk.classify.scikitlearn`

Estimators are the building block of scikit-learn. Almost everything is an estimator. All estimators have fit() methods. Most have either a predict() or transform() method. Supervised techniques generally have a score() method as well.

The basic ML workflow looks like this:

from sklearn.EstimatorFamily import Estimator
est = Estimator(hyperparameter_1, ... ,hyperparameter_n) # Create a model
est.fit(X_train, y_train) # Fit the model
est.score(X_test, y_test) # Evaluate model efficacy
est.predict(X_test) # Create predictions

## Importing estimators

`from sklearn.EstimatorFamily import Estimator` is typically how you'd import an estimator.  Some examples are:

```python
from sklearn.linear_model import RidgeRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
```

## OLS

Order of matrices is different than OLS.

In [23]:
from sklearn.linear_model import LinearRegression

lm_1 = LinearRegression().fit(X, y)

In [24]:
lm_1.score(X, y)
lm_1.coef_
lm_1.intercept_

array([-1059.02598083])

scikit learn will default to including an intercept. To get rid of it, we need to create a new method object and fit again.

In [25]:
lm_2 = LinearRegression(fit_intercept = False).fit(X, y)
lm_2.score(X, y)
fit_ols_cl_2.rsquared

0.8526976159298703

Why is r^2 different? The parameters are the same. The issue is that we have a linear model without an intercept. The reasons are to do with math. Zzzzz

In [26]:
lm_2.coef_
fit_ols_cl_2.params

prior_cumulative_paid:lag_cat[10]    0.008844
prior_cumulative_paid:lag_cat[2]     1.156867
prior_cumulative_paid:lag_cat[3]     0.328289
prior_cumulative_paid:lag_cat[4]     0.146553
prior_cumulative_paid:lag_cat[5]     0.086466
prior_cumulative_paid:lag_cat[6]     0.053993
prior_cumulative_paid:lag_cat[7]     0.040150
prior_cumulative_paid:lag_cat[8]     0.026309
prior_cumulative_paid:lag_cat[9]     0.023028
dtype: float64

If it puts your mind at ease, we can create an OLS fit and compare to the scikit learn.

In [27]:
fit_ols_cl_inter = smf.ols(
  'incremental_paid ~ 1 + prior_cumulative_paid:lag_cat', 
  data = df_allstate.dropna()
).fit()

fit_ols_cl_inter.rsquared
lm_1.score(X, y)

0.7859835045522741

Parting thoughts about $r^2$:

* Mostly used for OLS, though we can "fake it" by calculating correlation between observations and predictions.
* Does not consider things like serial correlation and outliers
* Absence of an intercept muddles the calculation in ways that your undergrad textbook doesn't describe
* Can be altered through a linear transform
* I think people like $r^2$ because it is an _absolute_, rather than a _relative_ measure of model quality.
  * When you get to GLMs, you can kiss $r^2$ good bye.
  * Low $r^2$ doesn't mean a _bad_ model. Also, high $r^2$ doesn't necessarily mean a _good_ model.
  * Metrics like RMSE, or MAE are useful ways to evaluate a model. How close your model resembles your data feels sort of important!

Moving on ...

Let's look at something a bit more like machine learning.

## Classifier

Pull in all lines.

In [28]:
lines = ['wkcomp', 'ppauto', 'comauto', 'medmal', 'prodliab', 'othliab']

df_triangles = []
for line in lines:
  df = fetch_triangle(line)
  df = augment_triangle(df)
  df_triangles.append(df)


This will basically do a row bind.

In [29]:
df_triangles = pd.concat(df_triangles, axis=0, ignore_index=False)

In [30]:
df_triangles.index.names
df_triangles.shape

(77900, 16)

Cheap and cheerful contingency table. Note that `othliab` is the plurality class.

In [31]:
df_triangles.group_name.groupby('line').count()

line
comauto     15800
medmal       3400
othliab     23900
ppauto      14600
prodliab     7000
wkcomp      13200
Name: group_name, dtype: int64

The target is just the `line`. It's in an index, so we'll need to use `get_level_values()` to extract it.

In [32]:
y = df_triangles.index.get_level_values('line')

`X` is simply a couple of columns

In [33]:
X = df_triangles[['ldf_paid', 'ldf_incurred']]

We have some data, let's classify. 

Split data into train and test

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
  X, y, 
  test_size = 0.25, 
  random_state = 1234)

## K-nearest neighbors

In [35]:
from sklearn.neighbors import KNeighborsClassifier

mdl_knn = KNeighborsClassifier()
mdl_knn.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

Need to impute for the weird values. Note that `SimpleImputer()` will NOT impute infinite values.

In [36]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(missing_values = np.nan, strategy = 'median')
imp.fit(X_train)

SimpleImputer(strategy='median')

How'd we do?

In [37]:
mdl_knn.fit(imp.transform(X_train), y_train)
mdl_knn.score(X_test, y_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

Forgot to impute!

In [38]:
mdl_knn.score(imp.transform(X_test), y_test)

0.3011039794608472

## Confusion matrix

In [39]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, mdl_knn.predict(imp.transform(X_test)), labels = lines)

array([[ 299,  392,  673,   36,   51, 1862],
       [ 263,  810, 1058,   23,   19, 1498],
       [ 238,  458, 1252,   23,   37, 1838],
       [  43,   49,  154,   17,   11,  581],
       [  53,   63,  397,   14,   40, 1194],
       [ 285,  342, 1799,   58,   99, 3446]], dtype=int64)

A confusion matrix is easy to convert to a data frame.

In [40]:
df_confusion_knn = pd.DataFrame(
  confusion_matrix(y_test, mdl_knn.predict(imp.transform(X_test)), labels = lines),
  index = lines, 
  columns = lines)

df_confusion_knn

,wkcomp,ppauto,comauto,medmal,prodliab,othliab
wkcomp,299,392,673,36,51,1862
ppauto,263,810,1058,23,19,1498
comauto,238,458,1252,23,37,1838
medmal,43,49,154,17,11,581
prodliab,53,63,397,14,40,1194
othliab,285,342,1799,58,99,3446


In [41]:
print(df_confusion_knn.sum())
print(df_confusion_knn.sum().sum())
print(X_test.shape)

wkcomp       1181
ppauto       2114
comauto      5333
medmal        171
prodliab      257
othliab     10419
dtype: int64
19475
(19475, 2)


## Logistic regression

In [42]:
from sklearn.linear_model import LogisticRegression

mdl_logistic = LogisticRegression() 
mdl_logistic.fit(imp.transform(X_train), y_train)

mdl_logistic.score(imp.transform(X_test), y_test)

0.3093196405648267

We did barely better than K-nearest neighbors.

In [43]:
df_confusion_logistic = pd.DataFrame(
  confusion_matrix(y_test, mdl_logistic.predict(imp.transform(X_test)), labels = lines),
  index = lines, 
  columns = lines)

df_confusion_logistic

,wkcomp,ppauto,comauto,medmal,prodliab,othliab
wkcomp,0,0,0,0,0,3313
ppauto,0,0,0,0,0,3671
comauto,0,0,0,0,0,3846
medmal,0,0,0,0,0,855
prodliab,0,4,0,0,2,1755
othliab,0,6,0,0,1,6022


Well. This is embarrassing.

## Maybe a subset of data?

In [44]:
lines_sub = ['medmal', 'ppauto', 'wkcomp']
df_tri_subs = df_triangles.query("line in ('medmal', 'ppauto', 'wkcomp')").copy()

y = df_tri_subs.index.get_level_values('line')
X = df_tri_subs[['ldf_paid', 'ldf_incurred']]

We have to impute again, because we have new data.

In [45]:
X_train, X_test, y_train, y_test = train_test_split(
  X, y, 
  test_size = 0.25, 
  random_state = 1234)

imp = SimpleImputer(missing_values = np.nan, strategy = 'median')
imp.fit(X_train)

SimpleImputer(strategy='median')

In [46]:
mdl_logistic.fit(imp.transform(X_train), y_train)
mdl_logistic.score(imp.transform(X_test), y_test)

df_confusion_logistic = pd.DataFrame(
  confusion_matrix(y_test, mdl_logistic.predict(imp.transform(X_test)), labels = lines_sub),
  index = lines_sub, 
  columns = lines_sub)

df_confusion_logistic

,medmal,ppauto,wkcomp
medmal,6,753,50
ppauto,0,3601,66
wkcomp,3,3194,127


## Feature engineering

Let's change things a bit. If we pivot the LDFs, perhaps we can detect some signal in the fact that LDFs depend on age.

In [47]:
df_tri_wide = df_tri_subs[['ldf_paid', 'ldf_incurred']].unstack('lag')
df_tri_wide.head()

df_tri_wide.columns

MultiIndex([(    'ldf_paid',  1),
            (    'ldf_paid',  2),
            (    'ldf_paid',  3),
            (    'ldf_paid',  4),
            (    'ldf_paid',  5),
            (    'ldf_paid',  6),
            (    'ldf_paid',  7),
            (    'ldf_paid',  8),
            (    'ldf_paid',  9),
            (    'ldf_paid', 10),
            ('ldf_incurred',  1),
            ('ldf_incurred',  2),
            ('ldf_incurred',  3),
            ('ldf_incurred',  4),
            ('ldf_incurred',  5),
            ('ldf_incurred',  6),
            ('ldf_incurred',  7),
            ('ldf_incurred',  8),
            ('ldf_incurred',  9),
            ('ldf_incurred', 10)],
           names=[None, 'lag'])

Our newly structured data frame looks good.

In [48]:
X = df_tri_wide.drop(1, axis = 1, level = 1)

y = df_tri_wide.index.get_level_values('line')

X_train, X_test, y_train, y_test = train_test_split(
  X, y, 
  test_size = 0.2, 
  random_state = 1234)
imp.fit(X_train)

SimpleImputer(strategy='median')

In [49]:
mdl_logistic.fit(imp.transform(X_train), y_train)

C:\Users\bfannin.CASACT\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Scale the data?! Ugh. 

In [50]:
mdl_knn.fit(imp.transform(X_train), y_train)
mdl_knn.score(imp.transform(X_test), y_test)

0.6314102564102564

Not too bad!

In [51]:
df_confusion_knn = pd.DataFrame(
  confusion_matrix(y_test, mdl_knn.predict(imp.transform(X_test))),
  index = lines_sub, 
  columns = lines_sub)

df_confusion_knn

,medmal,ppauto,wkcomp
medmal,36,29,7
ppauto,2,246,45
wkcomp,8,139,112


## Pipelines

Has a similar feel to the `recipes` package in R.

In [52]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
  transformers=[
    ('num', numeric_transformer, numeric_features)
  ]
)

In [53]:
mdl_logistic = Pipeline(
  steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
  ]
)

mdl_knn = Pipeline(
  steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
  ]
)

Pipeline looks pretty in HTML

In [54]:
from sklearn import set_config
set_config(display='diagram')
mdl_knn

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  MultiIndex([(    'ldf_paid',  2),
            (    'ldf_paid',  3),
            (    'ldf_paid',  4),
            (    'ldf_paid',  5),
            (    'ldf_paid',  6),
            (    'ldf_paid',  7),
            (    'ldf_paid',  8),
            (    'ldf_paid',  9),
            (    'ldf_paid', 10),
            ('ldf_incurred',  2),
            ('ldf_incurred',  3),
            ('ldf_incurred',  4),
            ('ldf_incurred',  5),
            ('ldf_incurred',  6),
            ('ldf_incurred',  7),
            ('ldf_incurred',  8),
            ('ldf_incurred',  9),
            ('ldf_incurred', 10)],
           names=[None, 'lag']))])),
                ('classifier', KNeighborsClassifier())])

Pretty, but how does it run?

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

mdl_logistic.fit(X_train, y_train)
mdl_logistic.score(X_test, y_test)

mdl_knn.fit(X_train, y_train)
mdl_knn.score(X_test, y_test)

C:\Users\bfannin.CASACT\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6137820512820513

## Cross validation

In [56]:
from sklearn.model_selection import cross_val_score

cross_val_score(mdl_knn, X, y, cv=5)
cross_val_score(mdl_logistic, X, y, cv=5)

array([0.58974359, 0.60737179, 0.58653846, 0.5849359 , 0.53525641])

<!-- TODO variable importance -->

## Wrapping up scikit-learn

* Almost everything is an Estimator. They all have a fit method and depending on the nature of the estimator may also have a predict, score or transform method.
* The API is standardized across estimator
* A transformer is a special type of estimator that tranforms data for another Estimator
* Cross-validation with Grid Search helps in hyperparameter selection
* Pipelines are useful for composing a chain of Estimators.
* The documentation is a goldmine of information